In [1]:
import json
import psycopg2
import pandas as pd
import numpy as np

In [2]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [10]:
db.rollback()

In [22]:
import datetime

def retrieve_taste_db(user_id,db):
  cursor = db.cursor()
  sql = f"SELECT taste,last_update from user_taste where user_id={user_id}"
  cursor.execute(sql)
  result = cursor.fetchone()
  return result

def calculate_taste(user_id,db):
  print("calculate_tast...")
  cursor = db.cursor()
  #1. retrieve from survey
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*22)
  theme_score = theme_score.reshape(1,-1)
  for place in survey_result:
    load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)*10
  #2. retrieve from place-like log
  load_like_log_sql = f'SELECT place_id FROM place_like WHERE member_id = {user_id};'
  cursor.execute(load_like_log_sql)
  user_like_log = cursor.fetchall()
  if user_like_log:
    user_like_log = list(map(lambda x: x[0],user_like_log))
    for place in user_like_log:
      load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
      cursor.execute(load_place_sql)
      place_theme = cursor.fetchone()
      theme_score += np.array(place_theme)*7
  
  #3. retrieve from place_click log
  load_click_log_sql = f'SELECT place_id FROM places_log WHERE member_id = {user_id};'
  cursor.execute(load_click_log_sql)
  user_click_log = cursor.fetchall()
  if user_click_log:
    user_click_log = list(map(lambda x: x[0],user_click_log))
    for place in user_click_log:
      load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
      cursor.execute(load_place_sql)
      place_theme = cursor.fetchone()
      theme_score += np.array(place_theme)
  print("calculate_tast DONE!!")
  print("user's taste : ",theme_score.reshape(1,-1))
  return theme_score.reshape(1,-1)

def retrieve_taste(user_id,db):
  info = retrieve_taste_db(user_id,db)
  cursor =db.cursor()
  if info != None: 
    if (datetime.date.today() - info[1].date()).days>=3:
      print("update calcuate")
      updated_taste = calculate_taste(user_id,db)
      print("updated_taste: ",updated_taste)
      db_updated_taste = list(map(int, updated_taste[0]))
      print("db_updated_taste : ",db_updated_taste)
      #sql = "update user_taste set taste = %s where user_id = %s,"(db_updated_taste,user_id)
      #update db
      cursor.execute("update user_taste set taste = %s, last_update = %s where user_id = %s",(db_updated_taste,datetime.datetime.now() , user_id))
      db.commit()
    else:
      print("db에서 그대로 가져와서 빨라용!!")
      return np.array(info[0]).reshape(1,-1)
  else:
    print("calculate new")
    updated_taste = calculate_taste(user_id,db)
    db_updated_taste = list(map(int, updated_taste[0]))
    #insert new row to db
    cursor.execute("INSERT INTO user_taste(user_id,taste) VALUES(%s, %s)",(user_id,db_updated_taste))
    db.commit()
    
  return updated_taste


In [7]:
datetime.datetime.now()

datetime.datetime(2022, 10, 11, 16, 39, 28, 111437)

In [23]:
user_id = 184
db= load_db()

taste = retrieve_taste(user_id,db)

update calcuate
calculate_tast...
[32, 19, 13, 6, 33, 8, 10, 25, 41, 72]
calculate_tast DONE!!
user's taste :  [[107  56 188 191 108  17  60 357 369 106  67  79 330 133 213 192  73  64
  411  67   8  56]]
updated_taste:  [[107  56 188 191 108  17  60 357 369 106  67  79 330 133 213 192  73  64
  411  67   8  56]]
db_updated_taste :  [107, 56, 188, 191, 108, 17, 60, 357, 369, 106, 67, 79, 330, 133, 213, 192, 73, 64, 411, 67, 8, 56]


In [15]:
db.rollback()

In [24]:
user_id = 184
db= load_db()

taste = retrieve_taste(user_id,db)

db에서 그대로 가져와서 빨라용!!


In [25]:
taste

array([[107,  56, 188, 191, 108,  17,  60, 357, 369, 106,  67,  79, 330,
        133, 213, 192,  73,  64, 411,  67,   8,  56]])

In [26]:
copy = taste.copy()
copy

array([[107,  56, 188, 191, 108,  17,  60, 357, 369, 106,  67,  79, 330,
        133, 213, 192,  73,  64, 411,  67,   8,  56]])

In [30]:
copy[0][0] *= 2

In [31]:
copy

array([[214,  56, 188, 191, 108,  17,  60, 357, 369, 106,  67,  79, 330,
        133, 213, 192,  73,  64, 411,  67,   8,  56]])

In [32]:
copy[0].max()

411

In [33]:
copy[0][1] = copy[0].max()

In [34]:
copy

array([[214, 411, 188, 191, 108,  17,  60, 357, 369, 106,  67,  79, 330,
        133, 213, 192,  73,  64, 411,  67,   8,  56]])

In [ ]:
# '자연경관' 0~3  2,3에 가중치 부여
# '실내' 4~7 4,5에 가중치 부여
# '실외' 4~7 6,7 에 가중치부여
# '힐링' 8~10 8,9에 가중치 부여
# '산책'8~10    8에 가중치 부여  12에 약간 부여
# '산'11~13 / 11에 가중치 부여
# '바다' 11~13 / 13에 가중치 부여
# '액티비티' 14~17 / 16에 약간 가중치 부여, 17에 가중치 크게 부여
# '인문학'  18~21  / 21에 가중치 부여
# '야경' / 적용 x

In [35]:
for a,b in ((2,1.5),(3,2.0)):
  print(a,b)

2 1.5
3 2.0


In [43]:
def weighing_taste(tags,user_taste):
  index_indicator = {'nature':[[2,1.5],[3,2.0]],'indoor':[[4,2.0],[5,1.5]],'outdoor':[[6,1.5],[7,2.0]],
                     'healing':[[8,2.0],[9,1.5]],'walking':[[8,2.0],[12,1.5]],'mountain':[[11,2.0]],
                     'sea':[[13,2.0]],'activity':[[16,1.5],[17,2.0]],'culture':[[21,2.0]]}
  #tags = ['nature','sea']
  #user_taste = array([[214, 411, 188, 191, 108,  17,  60, 357, 369, 106,  67,  79, 330, 133, 213, 192,  73,  64, 411,  67,   8,  56]])
  for tag in tags:
    print(index_indicator[tag])
    for idx,weight in index_indicator[tag]:
      user_taste[0][idx] *= weight


In [42]:
for a,b in [[13,2.0]]:
  print(a,b)

13 2.0


In [46]:
copy = taste.copy()
copy

array([[107,  56, 188, 191, 108,  17,  60, 357, 369, 106,  67,  79, 330,
        133, 213, 192,  73,  64, 411,  67,   8,  56]])

In [50]:
423 * 1.5

634.5

In [49]:
tag_list = ['nature','sea']
weighing_taste([tag_list],copy)
copy

[[2, 1.5], [3, 2.0]]
[[13, 2.0]]


array([[ 107,   56,  634, 1528,  108,   17,   60,  357,  369,  106,   67,
          79,  330, 1064,  213,  192,   73,   64,  411,   67,    8,   56]])

###태그가 없는경우

In [51]:
tag_list = ['nature','sea']
weighing_taste([],copy)
copy

array([[ 107,   56,  634, 1528,  108,   17,   60,  357,  369,  106,   67,
          79,  330, 1064,  213,  192,   73,   64,  411,   67,    8,   56]])